In [48]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

# 1. Análisis Sociodemográfico de Pacientes de Consultas

En este análisis se quiere analizar el perfil sociodemográfico de los pacientes que vienen a
realizar una consulta ambulatoria al INT. Se quieren resolver las siguientes preguntas:

- ¿Cuál es el rango etario de los pacientes atendidos?
- ¿Cuál es el sexo de los pacientes atendidos?
- ¿Cuál es la previsión de los pacientes atendidos?
- ¿Cuál es la región/comuna de los pacientes atendidos?
- ¿Cuál es el servicio de salud de los pacientes atendidos?

Además, se quiere responder cada unad e estas preguntas para cada una de las especialidades de
las consultas, diagnósticos y por tipo de consulta.

In [50]:
df_track = pd.read_csv("../data/processed/datos_limpios_track.csv", sep=";", encoding="latin-1")
df_his = pd.read_csv(
    "../data/processed/datos_limpios_diagnosticos.csv", sep=";", encoding="latin-1"
)

In [51]:
df_track["hora_completa_cita"] = pd.to_datetime(df_track["hora_completa_cita"])
df_his["fecha_reserva"] = pd.to_datetime(df_his["fecha_reserva"])

In [52]:
print(
    f"La base de Track tiene citas desde {df_track['hora_completa_cita'].min()} "
    f"a {df_track['hora_completa_cita'].max()}"
)

print(
    f"La base de HIS tiene citas desde {df_his['fecha_reserva'].min()} a "
    f"{df_his['fecha_reserva'].max()}"
)
print()
print(f"La base de Track tiene {df_track.shape[0]} citas")
print(f"La base de HIS tiene {df_his.shape[0]} citas")


La base de Track tiene citas desde 2016-01-04 08:00:00 a 2022-12-30 20:30:00
La base de HIS tiene citas desde 2016-01-04 08:00:00 a 2022-12-30 14:50:00

La base de Track tiene 758350 citas
La base de HIS tiene 265441 citas


In [53]:
datos_pacientes_unicos_por_sesion = df_track.groupby(["id_paciente", "hora_completa_cita"]).head(1)

In [54]:
consultas_atendidas_con_datos_sociodemograficos = pd.merge(
    df_his,
    datos_pacientes_unicos_por_sesion,
    how="left",
    left_on=["id_paciente", "fecha_reserva"],
    right_on=["id_paciente", "hora_completa_cita"],
)

sin_datos_en_track = consultas_atendidas_con_datos_sociodemograficos[
    consultas_atendidas_con_datos_sociodemograficos.estadocita.isna()
]

In [56]:
print(
    f"Luego de unir la base de HIS con Track, la primera ha quedado con "
    f"{consultas_atendidas_con_datos_sociodemograficos.shape[0]} consultas. "
    f"Hay {sin_datos_en_track.shape[0]} consultas de HIS sin información en Track"
)

Luego de unir la base de HIS con Track, la primera ha quedado con 265441 consultas. Hay 189 consultas de HIS sin información en Track
